In [23]:
# your code here
import numpy as np
import pandas as pd

from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, RandomizedSearchCV
from sklearn.metrics import mean_squared_error, r2_score

# Transformers
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import PolynomialFeatures, StandardScaler, OneHotEncoder
from sklearn.feature_selection import SelectPercentile, f_regression
from sklearn.compose import ColumnTransformer

# Models
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression

# Datasets / synthetic data
from sklearn.datasets import load_diabetes, make_regression

In [24]:
## Load dataset
df = pd.read_csv(
    'https://raw.githubusercontent.com/rhodes-byu/stat-486/main/data/OceanicFisheries/ocean_data.csv'
)
df.head()

,T_degC,Salnty,O2ml_L,Depthm,Bottom_D,Wind_Spd,Dry_T,Wet_T,Wea,Cloud_Typ,Cloud_Amt,Visibility
0,16.830,33.8510,5.560,65,1337.0,14.0,16.5,15.5,NaN,NaN,NaN,NaN
1,9.262,33.8481,2.729,140,1202.0,5.0,15.0,13.0,Partly Cloudy,Stratus,1/10 or less but not zero,10km to 20km
2,15.390,33.4260,5.990,30,3871.0,10.0,18.8,17.6,Continuous blowing snow,Stratus,10/10,10km to 20km
3,14.540,32.9470,5.840,42,4018.0,14.0,16.9,16.1,Continuous blowing snow,Stratocumulus,10/10,4km to 10km
4,7.410,34.1810,1.000,300,4058.0,21.0,16.3,14.9,Partly Cloudy,Stratocumulus,7/10 to 8/10,4km to 10km


In [25]:
## exclude non-numeric columns 'Wea', 'Cloud_Typ', 'Cloud_Amt', and 'Visibility'
df = df.select_dtypes(include=[np.number])
df.head()

,T_degC,Salnty,O2ml_L,Depthm,Bottom_D,Wind_Spd,Dry_T,Wet_T
0,16.830,33.8510,5.560,65,1337.0,14.0,16.5,15.5
1,9.262,33.8481,2.729,140,1202.0,5.0,15.0,13.0
2,15.390,33.4260,5.990,30,3871.0,10.0,18.8,17.6
3,14.540,32.9470,5.840,42,4018.0,14.0,16.9,16.1
4,7.410,34.1810,1.000,300,4058.0,21.0,16.3,14.9


In [26]:
## train-test split
X = df.drop(columns='T_degC')
y = df['T_degC']


X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, random_state=307)


In [27]:
## 2. Pipelines
# a.
pipe_linreg = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="mean")),
    ("features", PolynomialFeatures(degree=2, include_bias=False)),
    ("scaler", StandardScaler()),
    ("linreg", LinearRegression()),
])

In [28]:
# b & c & d

pipe_linreg.fit(X_train, y_train)
pred = pipe_linreg.predict(X_test)

print("Train MSE:", mean_squared_error(y_train, pipe_linreg.predict(X_train)))
print("Test MSE:", mean_squared_error(y_test, pred))

Train MSE: 1.5350956362420867
Test MSE: 1.754330861094824


In [29]:
# e
print("Variance of y_test:", np.var(y_test))

Variance of y_test: 14.8162872136925


In [30]:
## 3. Feature Importance
# a & b & c

betas = pipe_linreg.named_steps['linreg'].coef_
features = pipe_linreg.named_steps['features'].get_feature_names_out()

##order features by the magnitude of their coefficients
feature_importance = pd.DataFrame({
    'Feature': features,
    'Coefficient': betas
})

feature_importance.sort_values(by='Coefficient', ascending=False, inplace=True)
feature_importance

,Feature,Coefficient
9,x0 x2,147.569766
12,x0 x5,35.153778
8,x0 x1,32.217666
13,x0 x6,8.948392
10,x0 x3,3.238598
19,x1 x6,2.673665
18,x1 x5,2.590504
11,x0 x4,1.990912
24,x2 x6,1.931986
27,x3 x5,1.922931


In [31]:
# d
intercept = pipe_linreg.named_steps['linreg'].intercept_
print(intercept)

11.607865265528682


In [32]:
# 4. Hyperparameter Tuning with Grid Search
# a. 
pipe_knn = Pipeline(steps=[
    ("imputer", SimpleImputer()),
    ("features", PolynomialFeatures(include_bias=False)),
    ("scaler", StandardScaler()),
    ("knn", KNeighborsRegressor()),
])

In [33]:
# b
params = {
  'imputer__strategy':('mean','median'), 
  'features__degree':[1,2,3],
  'knn__n_neighbors':list(range(5, 105, 5)),
  'knn__weights':('uniform','distance')
}

gs = GridSearchCV(pipe_knn, param_grid=params, scoring='neg_mean_squared_error', cv=10)
gs.fit(X_train, y_train)

GridSearchCV(cv=10,
             estimator=Pipeline(steps=[('imputer', SimpleImputer()),
                                       ('features',
                                        PolynomialFeatures(include_bias=False)),
                                       ('scaler', StandardScaler()),
                                       ('knn', KNeighborsRegressor())]),
             param_grid={'features__degree': [1, 2, 3],
                         'imputer__strategy': ('mean', 'median'),
                         'knn__n_neighbors': [5, 10, 15, 20, 25, 30, 35, 40, 45,
                                              50, 55, 60, 65, 70, 75, 80, 85,
                                              90, 95, 100],
                         'knn__weights': ('uniform', 'distance')},
             scoring='neg_mean_squared_error')

In [34]:
# c
print("Best Hyperparameters:", gs.best_params_)
print("Best MSE:", -gs.best_score_)

Best Hyperparameters: {'features__degree': 3, 'imputer__strategy': 'median', 'knn__n_neighbors': 10, 'knn__weights': 'distance'}
Best MSE: 1.1942865460094287


In [35]:
# e & f
best_pipe = gs.best_estimator_
y_pred = best_pipe.predict(X_test)
test_mse = mean_squared_error(y_test, y_pred)
print("Test MSE with best hyperparameters:", test_mse)


Test MSE with best hyperparameters: 1.239843724505285


In [36]:
## 5. Randomized Grid Search
# b
rs = RandomizedSearchCV(pipe_knn, param_distributions=params, scoring='neg_mean_squared_error', cv=10)
rs.fit(X_train, y_train)

RandomizedSearchCV(cv=10,
                   estimator=Pipeline(steps=[('imputer', SimpleImputer()),
                                             ('features',
                                              PolynomialFeatures(include_bias=False)),
                                             ('scaler', StandardScaler()),
                                             ('knn', KNeighborsRegressor())]),
                   param_distributions={'features__degree': [1, 2, 3],
                                        'imputer__strategy': ('mean', 'median'),
                                        'knn__n_neighbors': [5, 10, 15, 20, 25,
                                                             30, 35, 40, 45, 50,
                                                             55, 60, 65, 70, 75,
                                                             80, 85, 90, 95,
                                                             100],
                                        'knn__weights': ('uniform',
                                                         'distance')},
                   scoring='neg_mean_squared_error')

In [37]:
# c
print("Best Hyperparameters:", rs.best_params_)
print("Best MSE:", -rs.best_score_)

Best Hyperparameters: {'knn__weights': 'distance', 'knn__n_neighbors': 15, 'imputer__strategy': 'mean', 'features__degree': 3}
Best MSE: 1.2735716787265094


In [38]:
# e & f
best_pipe = rs.best_estimator_
y_pred = best_pipe.predict(X_test)
test_mse = mean_squared_error(y_test, y_pred)
print("Test MSE with best hyperparameters:", test_mse)

Test MSE with best hyperparameters: 1.3180891652149933


In [39]:
## 6. Advanced Pipelines
df = pd.read_csv(
    'https://raw.githubusercontent.com/rhodes-byu/stat-486/main/data/OceanicFisheries/ocean_data.csv'
)
df.head()

,T_degC,Salnty,O2ml_L,Depthm,Bottom_D,Wind_Spd,Dry_T,Wet_T,Wea,Cloud_Typ,Cloud_Amt,Visibility
0,16.830,33.8510,5.560,65,1337.0,14.0,16.5,15.5,NaN,NaN,NaN,NaN
1,9.262,33.8481,2.729,140,1202.0,5.0,15.0,13.0,Partly Cloudy,Stratus,1/10 or less but not zero,10km to 20km
2,15.390,33.4260,5.990,30,3871.0,10.0,18.8,17.6,Continuous blowing snow,Stratus,10/10,10km to 20km
3,14.540,32.9470,5.840,42,4018.0,14.0,16.9,16.1,Continuous blowing snow,Stratocumulus,10/10,4km to 10km
4,7.410,34.1810,1.000,300,4058.0,21.0,16.3,14.9,Partly Cloudy,Stratocumulus,7/10 to 8/10,4km to 10km


In [40]:
X = df.drop(columns='T_degC')
y = df['T_degC']


X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, random_state=307)

In [41]:
numeric_features = ["Salnty", "O2ml_L", "Depthm", "Bottom_D", "Wind_Spd", "Dry_T", "Wet_T"]
categorical_features = ["Wea", "Cloud_Typ", "Cloud_Amt", "Visibility"]

In [42]:
numeric_pipe = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="mean")),
    ("polynomial", PolynomialFeatures(degree=2, include_bias=False)),
    ("scaler", StandardScaler()),
])

categorical_pipe = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("onehot", OneHotEncoder(sparse_output=False, handle_unknown="ignore")),
    ("selector", SelectPercentile(f_regression, percentile=50)),
])

preprocess = ColumnTransformer(
    transformers=[
        ("num", numeric_pipe, numeric_features),
        ("cat", categorical_pipe, categorical_features),
    ]
)

pipe_mixed = Pipeline(steps=[
    ("preprocess", preprocess),
    ("knn", KNeighborsRegressor(n_neighbors=20, weights='distance')),
])

In [43]:
# a
pipe_mixed.fit(X_train, y_train)
y_pred = pipe_mixed.predict(X_test)

test_mse = mean_squared_error(y_test, y_pred)
train_mse = mean_squared_error(y_train, pipe_mixed.predict(X_train))

print("Test MSE of advanced pipeline:", test_mse)
print("Train MSE of advanced pipeline:", train_mse)

Test MSE of advanced pipeline: 1.4623776711365377
Train MSE of advanced pipeline: 6.635422110145437e-13


In [ ]:
## 8. Share your model
# b
import joblib
joblib.dump(gs, 'lab-03-final-model.joblib', compress=3)